# GR-Only Bayesian Inference on GW170817
This notebook runs an MCMC fit using a simple GR-only model (no Curvality fields) on the real residuals from GW170817.

In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload Curvality_GR_Model_Colab_Files.zip
!unzip -o Curvality_GR_Model_Colab_Files.zip

In [ ]:
!pip install pymc arviz

In [ ]:
import pandas as pd
import pymc as pm
import arviz as az
import numpy as np
import matplotlib.pyplot as plt

# Load residuals
residuals = pd.read_csv('curvality_residuals.csv')
y = residuals['residuals'].values
t = np.arange(len(y)) * 1.0 / 4096  # Assuming 4096 Hz sample rate

In [ ]:
with pm.Model() as gr_model:
    sigma = pm.HalfNormal('sigma', sigma=1.0)
    mu = pm.Normal('mu', mu=0.0, sigma=1.0)
    y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=y)

    trace_gr = pm.sample(2000, tune=1000, target_accept=0.9, return_inferencedata=True)

In [ ]:
az.plot_trace(trace_gr);
plt.savefig('gr_trace.png', dpi=300)
summary = az.summary(trace_gr)
summary.to_csv('gr_summary.csv')
summary

In [ ]:
# If you already ran the Curvality model and have the trace:
# from arviz import compare
# az.compare({'GR': trace_gr, 'Curvality': trace_curvality})

In [ ]:
import shutil
from google.colab import files
shutil.make_archive('GR_Inference_Results', 'zip', '.', '.')
files.download('GR_Inference_Results.zip')